In [1]:
# Erasmus+ ICCT project (2018-1-SI01-KA203-047081)

# Toggle cell visibility

from IPython.display import HTML
tag = HTML('''<script>
code_show=true; 
function code_toggle() {
    if (code_show){
        $('div.input').hide()
    } else {
        $('div.input').show()
    }
    code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
Toggle cell visibility <a href="javascript:code_toggle()">here</a>.''')
display(tag)

# Hide the code completely

# from IPython.display import HTML
# tag = HTML('''<style>
# div.input {
#     display:none;
# }
# </style>''')
# display(tag)


## Osservabilità

Dato un sistema lineare del tipo

\begin{cases}
\dot{\textbf{x}}=A\textbf{x}+B\textbf{u} \\ 
\textbf{y}=C\textbf{x}
\end{cases}

si vuole sapere se, misurando la cronologia temporale dell'output $y(t)$ e dell'input $u(t)$, è possibile distinguere lo stato iniziale del sistema che ha generato l'output misurato.
Questo tipo di informazioni sul sistema è correlato alla proprietà chiamata **Osservabilità**.

Prima di tutto si introduce il concetto di <u>indistinguibilità</u> per due stati.
> Due stati $\bar{x_1}$ e $\hat{x_1}$ sono indistinguibili nel futuro (qui $\bar{x_1}I_{t_2}\hat{x_1}$) _se e solo se_ $\forall u \in U \quad y(t,t_1,\bar{x_1},u)=y(t,t_1,\hat{x_1},u) \quad \forall t\in [t_1,t_2]$. Dove $U$ è l'insieme dei valori di input.

In altre parole: due stati sono indistinguibili nel futuro se e solo se il sistema, partendo da essi ed evolvendo con lo stesso input $u(t)$ applicato, produce esattamente lo stesso output $y(t)$ nella finestra temporale considerata. Si può dimostrare che per i sistemi lineari tempo invarianti a tempo continuo, l'indistinguibilità (e quindi l'osservabilità) è indipendente dalla finestra temporale.

Vale la pena notare che l'indistinguibilità tra due stati (tra $x_1$ e $x_2$) è uguale all'indistinguibilità per la differenza dei due stati con l'origine dello spazio degli stati (tra $x_1-x_2$ e $0$).
Matematicamente, equiparando la risposta in uscita a partire dalle due condizioni iniziali si ottiene:

\begin{cases}
\bar{y}(t) = Ce^{At}\bar{x_1} + C\int_{t_1}^{t}e^{A(t-\tau)}Bu(\tau)d\tau \\ 
\hat{y}(t) = Ce^{At}\hat{x_1} + C\int_{t_1}^{t}e^{A(t-\tau)}Bu(\tau)d\tau
\end{cases}

$$
Ce^{At}\bar{x_1} = Ce^{At}\hat{x_1} \rightarrow Ce^{At}\underbrace{(\bar{x_1}-\hat{x_1})}_{\widetilde{x_1}} = Ce^{At}0 \,.
$$

Ciò implica che si può parlare di indistinguibilità dall'origine e, se uno stato è indistinguibile dall'origine allora si dice che è inosservabile.

Una misura di osservabilità di un sistema può essere introdotta utilizzando il _Gramiano di osservabilità_ $G_o(0,t_f)=\int_{0}^{t_f}e^{A^T\tau}C^TCe^{A\tau}d\tau$. Si può dimostrare che lo spazio degli stati non osservabili di un sistema è lo spazio nullo di $G_o$; quindi, un sistema lineare è osservabile se e solo se $\text{ker}(G_o)=\{0\}$ o, equivalentemente se è vero che $\widetilde{x}I_{t}0 \Rightarrow \widetilde{x}=\{0\}$.

Definendo la matrice di osservabilità come
$$
\mathcal{O} = \begin{bmatrix}C \\ CA \\ CA^2 \\ \vdots \\ CA^{n-1} \end{bmatrix}
$$
si può dimostrare che $\text{ker}(G_o)=\text{ker}(\mathcal{O})$ quindi, l'osservabilità di un sistema può essere investigata studiando il rango della matrice di osservabilità, cioè se $\text{rank}(\mathcal{O})=n$ dove $n: \, x\in \mathbb{R}^n$ allora si dice che il sistema è osservabile.

La conoscenza dell'osservabilità di un sistema e dei singoli stati è fondamentale in quanto discirmina quali modi del sistema compaiono in uscita e quali no. Se un sistema è osservabile, tutti i suoi modi possono apparire in uscita a seconda delle condizioni iniziali e delle funzioni di input. Se un sistema non è osservabile, cioè se $\mathcal{O}$ non ha rango pieno, allora alcuni modi del sistema non possono essere visti in uscita.

**N.B.:** Un concetto analogo all'osservabilità è la _ricostruibilità_. Ciò deriva dalla richiesta di poter ricostruire lo stato finale in cui si trova il sistema invece dello stato iniziale. Per i sistemi lineari a tempo continuo, un sistema è ricostruibile se e solo se è osservabile, a tempo discreto questa proprietà non vale più. Ciò può essere facilmente dimostrato realizzando che una volta che lo stato iniziale è stato _osservato_, lo stato corrente può essere _ricostituito_ utilizzando l'equazione di Lagrange (poiché l'input $u$ è noto). Viceversa, se lo stato corrente è noto, lo stato iniziale può essere ottenuto integrando indietro nel tempo.

### Come usare questo notebook?

Di seguito viene presentato un esempio interattivo in cui è possibile modificare le matrici $A$ e $C$.
- Cerca di rendere inosservabile, se possibile, solo il terzo stato $x_1$ della coppia predefinita $(A,C)$ modificando solo $C$.
- Cerca di rendere inosservabile, se possibile, lo stato $x_2$ della coppia predefinita $(A,C)$ modificando solo $C$.
- Prova a rendere inosservabile la coppia $(A,C)$ modificando solo $C$ e lasciando inalterata $A$. Una volta fatto, pensa alla struttura di $A$ e rispondi: perché il sistema è diventato inosservabile? Per quale(i) stato(i) ha perso l'osservabilità?
- Con $C=[1, 1, 1]$ cambia $A$ in modo da rendere $(A,C)$ non osservabile (aiutati pensando alla matrice $\mathcal{O}$).
- Prova a scrivere una coppia $(A,C)$ che abbia un autovalore positivo associato a uno stato non osservabile. Come si comporta l'output nella risposta libera?

In [2]:
#Preparatory Cell 

%matplotlib inline
import control
import numpy
from IPython.display import display, Markdown
import ipywidgets as widgets
import matplotlib.pyplot as plt

#print a matrix latex-like
def bmatrix(a):
     """Returns a LaTeX bmatrix - by Damir Arbula (ICCT project)

     :a: numpy array
     :returns: LaTeX bmatrix as a string
     """
     if len(a.shape) > 2:
         raise ValueError('bmatrix can at most display two dimensions')
     lines = str(a).replace('[', '').replace(']', '').splitlines()
     rv = [r'$\begin{bmatrix}']
     rv += ['  ' + ' & '.join(l.split()) + r'\\' for l in lines]
     rv +=  [r'\end{bmatrix}$']
     return ''.join(rv) #'\n'.join(rv)

#matrixWidget is a matrix looking widget built with a VBox of HBox(es) that returns a numPy array as value !
class matrixWidget(widgets.VBox):
    def updateM(self,change):
        for irow in range(0,self.n):
            for icol in range(0,self.m):
                self.M_[irow,icol] = self.children[irow].children[icol].value
                #print(self.M_[irow,icol])
        self.value = self.M_

    def dummychangecallback(self,change):
        pass
    
    
    def __init__(self,n,m):
        self.n = n
        self.m = m
        self.M_ = numpy.matrix(numpy.zeros((self.n,self.m)))
        self.value = self.M_
        widgets.VBox.__init__(self,
                             children = [
                                 widgets.HBox(children = 
                                              [widgets.FloatText(value=0.0, layout=widgets.Layout(width='90px')) for i in range(m)]
                                             ) 
                                 for j in range(n)
                             ])
        
        #fill in widgets and tell interact to call updateM each time a children changes value
        for irow in range(0,self.n):
            for icol in range(0,self.m):
                self.children[irow].children[icol].value = self.M_[irow,icol]
                self.children[irow].children[icol].observe(self.updateM, names='value')
        #value = Unicode('example@example.com', help="The email value.").tag(sync=True)
        self.observe(self.updateM, names='value', type= 'All')
        
    def setM(self, newM):
        #disable callbacks, change values, and reenable
        self.unobserve(self.updateM, names='value', type= 'All')
        for irow in range(0,self.n):
            for icol in range(0,self.m):
                self.children[irow].children[icol].unobserve(self.updateM, names='value')
        self.M_ = newM
        self.value = self.M_
        for irow in range(0,self.n):
            for icol in range(0,self.m):
                self.children[irow].children[icol].value = self.M_[irow,icol]
        for irow in range(0,self.n):
            for icol in range(0,self.m):
                self.children[irow].children[icol].observe(self.updateM, names='value')
        self.observe(self.updateM, names='value', type= 'All')        

                #self.children[irow].children[icol].observe(self.updateM, names='value')

             
#overlaod class for state space systems that DO NOT remove "useless" states (what "professor" of automatic control would do this?)
class sss(control.StateSpace):
    def __init__(self,*args):
        #call base class init constructor
        control.StateSpace.__init__(self,*args)
    #disable function below in base class
    def _remove_useless_states(self):
        pass

In [3]:
# Preparatory cell
A = numpy.matrix('-1 1 0; 0 -1 1; 0 0 -1')
B = numpy.matrix('0; 0; 1')
C = numpy.matrix('1 0 0')
X0 = numpy.matrix('2; 2; 2')

Aw = matrixWidget(3,3)
Aw.setM(A)
Bw = matrixWidget(3,1)
Bw.setM(B)
Cw = matrixWidget(1,3)
Cw.setM(C)
X0w = matrixWidget(3,1)
X0w.setM(X0)

In [4]:
# Misc
#define type of ipout 
selu = widgets.Dropdown(
    options=[('impulso','impulse'), ('gradino','step'), ('sinusoide','sinusoid'), ('onda quadra','square wave')],
    value='step',
    description='Input:',
    disabled=False
)
# Define the values of the input
u = widgets.FloatSlider(
    value=0,
    min=0,
    max=20.0,
    step=0.1,
    description=r'$u$:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)
period = widgets.FloatSlider(
    value=1,
    min=0.01,
    max=4,
    step=0.01,
    description='Periodo: ',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.2f',
)

#create dummy widget 
DW = widgets.FloatText(layout=widgets.Layout(width='0px', height='0px'))

#create button widget
START = widgets.Button(
    description='Test',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Test',
    icon='check'
)
                       
def on_start_button_clicked(b):
    #This is a workaround to have intreactive_output call the callback:
    #   force the value of the dummy widget to change
    if DW.value> 0 :
        DW.value = -1
    else: 
        DW.value = 1
    pass
START.on_click(on_start_button_clicked)

In [7]:
def main_callback(A, B, C, X0, u, period, selu, DW):
    eig = numpy.linalg.eig(A)[0]
    O = control.obsv(A,C)
    rankO = numpy.linalg.matrix_rank(O)
    
    text = r'Gli autovalori del sistema sono ' + str(eig[0]) + r', ' + str(eig[1])+ r' e ' + str(eig[2]) + r'. '
    if rankO == 3:
        text = text + r'Dato che $\text{rank}(\mathcal{O})=3$ il sistema è osservabile. ' + '\n\n' + 'La matrice di osservabilità è: ' + r'$\mathcal{O}=$' + bmatrix(O) #+ r'$'
    else:
        text = text + r'Dato che $\text{rank}(\mathcal{O})=' + str(rankO) + r'$ il sistema è inosservabile. ' + '\n\n' + 'La matrice di osservabilità è: ' + r'$\mathcal{O}=' + bmatrix(O) + r'$'
    display(Markdown(text))
    
    
    sys = sss(A,B,C,0)
    if numpy.real([eig[0],eig[1],eig[2]])[0] == 0 and numpy.real([eig[0],eig[1],eig[2]])[1] == 0 and numpy.real([eig[0],eig[1],eig[2]])[2] == 0:
        T = numpy.linspace(0,20,1000)
    else:
        if min(numpy.abs(numpy.real([eig[0],eig[1],eig[2]]))) != 0:
            T = numpy.linspace(0,7*1/min(numpy.abs(numpy.real([eig[0],eig[1],eig[2]]))),1000)
        else:
            T = numpy.linspace(0,7*1/max(numpy.abs(numpy.real([eig[0],eig[1],eig[2]]))),1000)
    
    if selu == 'impulse': #selu
        U = [0 for t in range(0,len(T))]
        U[0] = u
        T, Y, X = control.forced_response(sys,T=T,U=U,X0=X0)
    if selu == 'step':
        U = [u for t in range(0,len(T))]
        T, Y, X = control.forced_response(sys,T=T,U=U,X0=X0)
    if selu == 'sinusoid':
        U = u*numpy.sin(2*numpy.pi/period*T)
        T, Y, X = control.forced_response(sys,T=T,U=U,X0=X0)
    if selu == 'square wave':
        U = u*numpy.sign(numpy.sin(2*numpy.pi/period*T))
        T, Y, X = control.forced_response(sys,T=T,U=U,X0=X0)
    
    fig = plt.figure(figsize=(16,5))
    
    fig.add_subplot(121)
    plt.title('Risposta: stati')
    plt.plot(T,X[0])
    plt.plot(T,X[1])
    plt.plot(T,X[2])
    plt.ylabel('Stati')
    plt.xlabel('t [s]')
    plt.legend([r'$x_1$',r'$x_2$',r'$x_3$'])
    plt.grid()
    
    fig.add_subplot(122)
    plt.ylabel(r'$y$')
    plt.plot(T,Y)
    plt.xlabel('t [s]')
    plt.title('Risposta: uscita')
    plt.grid()
    
    
alltogether = widgets.VBox([widgets.HBox([widgets.Label('A:',border=3),  Aw, widgets.Label('   ',border=3),
                                          widgets.Label('B:',border=3),  Bw, widgets.Label('   ',border=3),
                                          widgets.Label('x0:',border=3),X0w,widgets.Label('   ',border=3),
                                          START]),
                            widgets.Label('   ',border=3),
                            widgets.HBox([widgets.Label('C:',border=3),Cw,widgets.Label('   ',border=3)]),
                            widgets.Label('   ',border=3),
                            widgets.HBox([selu, u, period])])
out = widgets.interactive_output(main_callback, {'A':Aw, 'B':Bw, 'C':Cw, 'X0':X0w, 'u':u, 'period':period, 'selu':selu, 'DW':DW})
out.layout.height = '460px'
display(out, alltogether)

Output(layout=Layout(height='460px'))

In [6]:
%%js
Jupyter.notebook.execute_cells([6])

<IPython.core.display.Javascript object>